# Open3D Demo and Registration
http://open3d.org/

In [125]:
from py3d import *
import numpy as np
import pandas as pd
import copy

### Point Cloud Visualization

In [126]:
ply_file = "new-data/1503619123211929000-pointcloud.ply"
pcd = read_point_cloud(ply_file)
draw_geometries([pcd])

### Voxel Downsampling

In [127]:
print("Downsample the point cloud with a voxel of 0.25")
downpcd = voxel_down_sample(pcd, voxel_size = 0.25)
draw_geometries([downpcd])

Downsample the point cloud with a voxel of 0.25


### Vertex Normal Estimation
Press `n` to see point normals and `-` and `+` to control the normal length

In [128]:
print("Recompute the normal of the downsampled point cloud")
estimate_normals(downpcd, search_param = KDTreeSearchParamHybrid(
    radius = 0.5, max_nn = 30))
draw_geometries([downpcd])
print("")

Recompute the normal of the downsampled point cloud



## Point Cloud Registration

### Cleaning Point Cloud Data
Removes noise near the lidar machine

In [129]:
file_path = "new-data/"
csv_name_1 = "1503619123211929083-cloudpoint.csv"
csv_name_2 = "1503619123412378073-cloudpoint.csv"

df_1 = pd.read_csv(file_path + csv_name_1, 
                 names = ['x', 'y', 'z', 
                          'intensity', 'ring', 'rotation', 'revolution'])
df_2 = pd.read_csv(file_path + csv_name_2, 
                 names = ['x', 'y', 'z', 
                          'intensity', 'ring', 'rotation', 'revolution'])

df_1['distance'] = np.sqrt(df_1['x']**2 + df_1['y']**2 + df_1['z']**2)
df_2['distance'] = np.sqrt(df_2['x']**2 + df_2['y']**2 + df_2['z']**2)

df_1 = df_1[df_1.distance > 2.5]
df_2 = df_2[df_2.distance > 2.5]

src_arr = df_1[['x', 'y', 'z']].as_matrix()
dest_arr = df_2[['x', 'y', 'z']].as_matrix()

In [130]:
src_ply_file_name = "src.ply"

with open(file_path + src_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(src_arr.shape[0]))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("end_header\n")
    
    for i in range(src_arr.shape[0]):
        f.write("{} {} {}\n".format(src_arr[i][0], 
                                    src_arr[i][1], 
                                    src_arr[i][2]))


dest_ply_file_name = "dest.ply"

with open(file_path + dest_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(dest_arr.shape[0]))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("end_header\n")
    
    for i in range(dest_arr.shape[0]):
        f.write("{} {} {}\n".format(dest_arr[i][0], 
                                    dest_arr[i][1], 
                                    dest_arr[i][2]))

In [131]:
source = read_point_cloud(file_path + "src.ply")
target = read_point_cloud(file_path + "dest.ply")

## Registration
`fitness` measures the overlapping area (higher is better)  
`inlier_rmse` measures the RMSE of all inlier correspondences (lower is better)

In [132]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    draw_geometries([source_temp, target_temp])

### Initial Alignment

In [133]:
threshold = 1
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 1.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])

In [134]:
print("Initial alignment")
evaluation = evaluate_registration(source, target, 
                                   threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness = 0.990676, inlier_rmse = 0.276300, and correspondence_set size of 117940
Access transformation to get result.


In [135]:
draw_registration_result(source, target, trans_init)

### Point-to-Point ICP

In [136]:
print("Apply point-to-point ICP")
reg_p2p = registration_icp(source, target, threshold, trans_init,
        TransformationEstimationPointToPoint(),
        ICPConvergenceCriteria(max_iteration = 100))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
print("")

Apply point-to-point ICP
RegistrationResult with fitness = 0.994759, inlier_rmse = 0.153341, and correspondence_set size of 118426
Access transformation to get result.
Transformation is:
[[  9.99999557e-01  -4.03438603e-04  -8.50647633e-04  -5.78615347e-01]
 [  4.03627829e-04   9.99999894e-01   2.22289364e-04   5.15516929e-02]
 [  8.50557863e-04  -2.22632611e-04   9.99999613e-01   2.55775331e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]



In [137]:
draw_registration_result(source, target, reg_p2p.transformation)

### Point-to-Plane ICP

In [138]:
print("Compute the normal")
estimate_normals(target, search_param = KDTreeSearchParamHybrid(
    radius = .5, max_nn = 30))
# draw_geometries([source])
print("")

Compute the normal



In [139]:
print("Apply point-to-plane ICP")
reg_p2l = registration_icp(source, target, threshold, trans_init,
        TransformationEstimationPointToPlane(),
        ICPConvergenceCriteria(max_iteration = 100))
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")

Apply point-to-plane ICP
RegistrationResult with fitness = 0.995052, inlier_rmse = 0.161588, and correspondence_set size of 118461
Access transformation to get result.
Transformation is:
[[  9.99999096e-01   1.95794685e-04  -1.33007300e-03  -6.86554470e-01]
 [ -1.95450538e-04   9.99999947e-01   2.58868243e-04   3.69696655e-03]
 [  1.33012362e-03  -2.58608045e-04   9.99999082e-01   4.17568269e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]



In [140]:
draw_registration_result(source, target, reg_p2l.transformation)